In [2]:
import pandas as pd
import os,shutil
import random
import numpy as np

In [7]:
File_path="chestxray_covid/metadata.csv"
Images_path="chestxray_covid/images"
df=pd.read_csv(File_path)
df.head()
df.columns


Index(['patientid', 'offset', 'sex', 'age', 'finding', 'RT_PCR_positive',
       'survival', 'intubated', 'intubation_present', 'went_icu', 'in_icu',
       'needed_supplemental_O2', 'extubated', 'temperature', 'pO2_saturation',
       'leukocyte_count', 'neutrophil_count', 'lymphocyte_count', 'view',
       'modality', 'date', 'location', 'folder', 'filename', 'doi', 'url',
       'license', 'clinical_notes', 'other_notes'],
      dtype='object')

In [10]:
os.mkdir("covid_dataset")
Target_dir="covid_dataset/covid"
if not os.path.exists(Target_dir):
    os.mkdir(Target_dir)

In [11]:
cnt=0
for(i,row) in df.iterrows():
    if row["finding"]=="COVID-19" and row["view"]=="PA":
        filename=row["filename"]
        image_path=os.path.join(Images_path,filename)
        image_copy_path=os.path.join(Target_dir,filename)
        shutil.copy2(image_path,image_copy_path)
        print("Moving image",cnt)
        cnt+=1
print(cnt)

Moving image 0
Moving image 1
Moving image 2
Moving image 3
Moving image 4
Moving image 5
Moving image 6
Moving image 7
Moving image 8
Moving image 9
Moving image 10
Moving image 11
Moving image 12
Moving image 13
Moving image 14
Moving image 15
Moving image 16
Moving image 17
Moving image 18
Moving image 19
Moving image 20
Moving image 21
Moving image 22
Moving image 23
Moving image 24
Moving image 25
Moving image 26
Moving image 27
Moving image 28
Moving image 29
Moving image 30
Moving image 31
Moving image 32
Moving image 33
Moving image 34
Moving image 35
Moving image 36
Moving image 37
Moving image 38
Moving image 39
Moving image 40
Moving image 41
Moving image 42
Moving image 43
Moving image 44
Moving image 45
Moving image 46
Moving image 47
Moving image 48
Moving image 49
Moving image 50
Moving image 51
Moving image 52
Moving image 53
Moving image 54
Moving image 55
Moving image 56
Moving image 57
Moving image 58
Moving image 59
Moving image 60
Moving image 61
Moving image 62
Mo

In [19]:
#sampling of images from kaggle dataset and normal xray images

In [14]:
Kaggle_file_path="chest_xray/train/NORMAL"
Target_Normal_Dir="covid_dataset/Normal"
images_names=os.listdir(Kaggle_file_path)
random.shuffle(images_names)

In [16]:
for i in range(205):
    image_name=images_names[i]
    image_path=os.path.join(Kaggle_file_path,image_name)
    target_path=os.path.join(Target_Normal_Dir,image_name)
    shutil.copy2(image_path,target_path)
    print("copying image",i)

copying image 0
copying image 1
copying image 2
copying image 3
copying image 4
copying image 5
copying image 6
copying image 7
copying image 8
copying image 9
copying image 10
copying image 11
copying image 12
copying image 13
copying image 14
copying image 15
copying image 16
copying image 17
copying image 18
copying image 19
copying image 20
copying image 21
copying image 22
copying image 23
copying image 24
copying image 25
copying image 26
copying image 27
copying image 28
copying image 29
copying image 30
copying image 31
copying image 32
copying image 33
copying image 34
copying image 35
copying image 36
copying image 37
copying image 38
copying image 39
copying image 40
copying image 41
copying image 42
copying image 43
copying image 44
copying image 45
copying image 46
copying image 47
copying image 48
copying image 49
copying image 50
copying image 51
copying image 52
copying image 53
copying image 54
copying image 55
copying image 56
copying image 57
copying image 58
copying

In [26]:
##COVID-19 Using Xrays

In [3]:
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

Using TensorFlow backend.


In [4]:
model=Sequential()

In [5]:

model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Convolution2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 54, 54, 64)       

In [7]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [8]:
train_datagen=image.ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen=image.ImageDataGenerator(rescale=1.0/255)

In [9]:
train_generator=train_datagen.flow_from_directory(
    "covid_dataset/Train",
    target_size=(224,224),
    batch_size=32,
    class_mode='binary')

Found 330 images belonging to 2 classes.


In [10]:
train_generator.class_indices

{'Normal': 0, 'covid': 1}

In [11]:
val_generator=val_datagen.flow_from_directory(
    "covid_dataset/Validation",
    target_size=(224,224),
    batch_size=32,
    class_mode='binary')

Found 80 images belonging to 2 classes.


In [26]:
history=model.fit_generator(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
11/11 [==============================] - 242s 22s/step - loss: 1.6128 - accuracy: 0.5242 - val_loss: 0.6919 - val_accuracy: 0.5000
Epoch 2/10
11/11 [==============================] - 220s 20s/step - loss: 0.6760 - accuracy: 0.5758 - val_loss: 0.6184 - val_accuracy: 0.9625
Epoch 3/10
11/11 [==============================] - 219s 20s/step - loss: 0.5352 - accuracy: 0.6939 - val_loss: 0.3819 - val_accuracy: 0.9750
Epoch 4/10
11/11 [==============================] - 213s 19s/step - loss: 0.4280 - accuracy: 0.8061 - val_loss: 0.1483 - val_accuracy: 0.9750
Epoch 5/10
11/11 [==============================] - 209s 19s/step - loss: 0.3789 - accuracy: 0.8515 - val_loss: 0.0358 - val_accuracy: 0.9875
Epoch 6/10
11/11 [==============================] - 206s 19s/step - loss: 0.2740 - accuracy: 0.9212 - val_loss: 0.0533 - val_accuracy: 0.9875
Epoch 7/10
11/11 [==============================] - 227s 21s/step - loss: 0.3455 - accuracy: 0.8636 - val_loss: 0.1761 - val_accuracy: 0.9750
Epoch 

In [12]:
count=0
for i in os.listdir("chest_xray/test/NORMAL"):
    img = image.load_img("chest_xray/test/NORMAL/"+i,target_size=(224,224))
    img= image.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    p=model.predict_classes(img)
    if(p!=0):
        count+=1
    print(p)
print(count)

[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]